In [1]:
import pandas as pd

from lib.tasks.legiscan import infer_resolver_map, augment_resolver_map

LEGISCAN_API_URL = 'https://api.legiscan.com/'
ACLU_DATA_PATH = '../datasets/aclu.json'
TTL_DATA_PATH = '../datasets/tracktranslegislation.json'
RESOLVER_MAP = '../artifacts/resolver_map.json'
HINT_MAP = '../configuration/resolver_hints.json'

new_mapper = infer_resolver_map(TTL_DATA_PATH, RESOLVER_MAP, HINT_MAP)
aclu_data = pd.read_json(ACLU_DATA_PATH)

augment_resolver_map(
    new_mapper, 
    [(row['state']['value'], row['name']) for idx, row in aclu_data.iterrows()], #  if row['state']['value']=='ID'
    RESOLVER_MAP,
)

Resolution failure: ('OH', 'SB 83') Unknown prefix S for OH SB 83
Resolution failure: ('TX', 'HB 5127') No relevant matches for TX HB 5127
Resolution failure: ('TX', 'HB 5261') No relevant matches for TX HB 5261
Resolution failure: ('MA', 'H. 458') Unknown state MA for MA H. 458
Resolution failure: ('MA', 'H. 509') Unknown state MA for MA H. 509
Resolution failure: ('TX', 'SB 17') No relevant matches for TX SB 17
Resolution failure: ('TX', 'SB 2199') No relevant matches for TX SB 2199
Resolution failure: ('TX', 'SB 8') No relevant matches for TX SB 8
Resolution failure: ('TX', 'HB 4624') No relevant matches for TX HB 4624
Resolution failure: ('TX', 'HB 3164') No relevant matches for TX HB 3164
Resolution failure: ('ME', 'HP 577') No relevant matches for ME HP 577
Resolution failure: ('ME', 'HP 447') No relevant matches for ME HP 447
Resolution failure: ('ID', 'HB 63') No relevant matches for ID HB 63
Resolution failure: ('IA', 'HF 9') No relevant matches for IA HF 9
